In [1]:
import normals_lib
import kaolin as kal
import torch
import numpy as np
import meshplot as mp

#mesh = kal.rep.TriangleMesh.from_obj("model.obj") # Modelo de una esfera
#mesh = kal.rep.TriangleMesh.from_obj("cow.obj") # Modelo de vaca
mesh = kal.rep.TriangleMesh.from_obj("torus.obj")
#mesh = kal.rep.TriangleMesh.from_obj("pear.obj")

In [2]:
normals = normals_lib.compute_simple_vertex_normals(mesh).to(device=torch.device('cpu'))
normals_lib.visualize_normals(mesh, normals)
normals = normals.to(device=torch.device('cuda'))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [3]:
# Calculate principal curvatures and curvature directions
import curvature

k1, k2, e1, e2 = curvature.compute_curvatures(mesh)

In [4]:
import dcurv

dc = dcurv.compute_dcurvs(mesh, curvs=(k1,k2,e1,e2))

In [7]:
import importlib
import shared_algs
importlib.reload(dcurv)
importlib.reload(shared_algs)

<module 'dcurv' from '/home/danniccs/rtsc_to_torch/dcurv.py'>

In [26]:
view_coords = torch.load("view_coordinates.pt").to(device=torch.device('cuda'))
verts = mesh.vertices.clone()
xmed = torch.median(verts[:,0])
ymed = torch.median(verts[:,1])
zmed = torch.median(verts[:,2])
verts[:,0] -= xmed
verts[:,1] -= ymed
verts[:,2] -= zmed
verts /= torch.max(torch.norm(verts, dim=1))
mesh.vertices = verts

In [27]:
DwKr, kr = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc, view_coords=view_coords[0])

In [47]:
tridc = torch.zeros(dc.shape)
with open("/home/danniccs/rtsc-1.6/dcurv", "r") as myfile:
    for i in range(0, dc.shape[0]):
        l = myfile.readline()
        tridc[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])
DwKrtri, krtri = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=tridc.to(device=torch.device('cuda')))

In [30]:
colour = torch.zeros(mesh.vertices.shape[0], 3)
#colour[:,0] = DwKr.to(device=torch.device('cpu'))
mp.plot(verts.numpy(), mesh.faces.numpy(), c=DwKr.to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet_r"})
mp.plot(verts.numpy(), mesh.faces.numpy(), c=kr.to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet_r"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [14]:
torch.max(mesh.vertices[:,0])

tensor(1.2500)